In [4]:
%%javascript

// ============================================================  //
// This part of the tutorial is for creating NDVI and dNDVI maps //
// ============================================================  //


// Specify center location and for GRSM
var SiteCenterPoint = ee.Geometry.Point([-83.5, 35.7]);

// Center the map on our area of interest and set zoom level
Map.setCenter(-83.5, 35.7, 12);

var study_polygon = /* color: #98ff00 */ee.Geometry.Polygon(
        [[[-83.5425714556907, 35.701602862979236],
          [-83.5422281329368, 35.690310616971246],
          [-83.53879490539774, 35.63828989061714],
          [-83.5037759844993, 35.615406781430224],
          [-83.48592430887022, 35.61938453875363],
          [-83.47459355041727, 35.627546807436595],
          [-83.47922808124717, 35.65117980654369],
          [-83.47738302453864, 35.682051678012314],
          [-83.4774257678571, 35.69326203209512],
          [-83.47751235961914, 35.71456632200802],
          [-83.47837061486136, 35.7182597023649],
          [-83.48043060302734, 35.72055934657099],
          [-83.53656387329102, 35.70494866628485]]]);

// Define the GRSM perim variable
var grsm_boundary = ee.FeatureCollection('projects/ee-stbr4432/assets/grsm_polygon');

// Define the fire perimeter variable
var fire_perimeter = ee.FeatureCollection('projects/ee-stbr4432/assets/chimney_tops_perim');
var fireBoundGeom = fire_perimeter.geometry();

// Apply the intersection method to the Polygon object.
var polygonIntersection = fireBoundGeom.intersection(grsm_boundary);
print('polygon intersection',polygonIntersection);

// Create study area.
var studyArea = study_polygon.intersection(polygonIntersection);

// Print polygon area in square kilometers.
print('Study area square: ', studyArea.area().divide(1000 * 1000));

// Print polygon perimeter length in kilometers.
print('Study area perimeter: ', studyArea.perimeter().divide(1000));


// ======= Read 2016 data and calculate NDVI ======= //


// Read in the 2016 SDR Image Collection at GRSM
var NISimage_2016 = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')
  .filterBounds(SiteCenterPoint)
  .filterDate('2016-06-08', '2016-06-30')
  .filterMetadata('NEON_SITE', 'equals', 'GRSM')
  .first()
  .clip(studyArea);

// 2016 pre-fire NDVI
var pre_fire_ndvi_2016 = NISimage_2016.normalizedDifference(['B090', 'B053']);
Map.addLayer(pre_fire_ndvi_2016, {min:0, max:1, palette:['pink', 'white', 'lightgreen', 'darkgreen']}, 'Pre Fire NDVI 2016');


// ======= Read 2017 data and calculate NDVI ======= //


// Read in the 2017 SDR Image Collection at GRSM
var NISimage_2017 = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')
  .filterBounds(SiteCenterPoint)
  .filterDate('2017-10-01', '2017-10-31')
  .filterMetadata('NEON_SITE', 'equals', 'GRSM')
  .first()
  .clip(studyArea);
  
// 2017 post-fire NDVI
var post_fire_ndvi_2017 = NISimage_2017.normalizedDifference(['B090', 'B053']);
Map.addLayer(post_fire_ndvi_2017, {min:0, max:1, palette:['pink', 'white', 'lightgreen', 'darkgreen']}, 'Post Fire NDVI 2017');


// ======= Read 2021 data and calculate NDVI ======= //


// Read in the 2021 SDR Image Collection at GRSM
var NISimage_2021 = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')
  .filterBounds(SiteCenterPoint)
  .filterDate('2021-06-18', '2021-06-26')
  .filterMetadata('NEON_SITE', 'equals', 'GRSM')
  .first()
  .clip(studyArea);
  
// 2021 post-fire recovery NDVI
var post_fire_ndvi_2021 = NISimage_2021.normalizedDifference(['B090', 'B053']);
Map.addLayer(post_fire_ndvi_2021, {min:0, max:1, palette:['pink', 'white', 'lightgreen', 'darkgreen']}, 'Post Fire recovery NDVI 2021');


// ======= Plot images of difference ======= //

var dNDVI_2017_2016 = post_fire_ndvi_2017.subtract(pre_fire_ndvi_2016);
var dNDVI_2021_2016 = post_fire_ndvi_2021.subtract(pre_fire_ndvi_2016);

Map.addLayer(dNDVI_2017_2016, {min:-1, max:1, palette:['pink', 'white', 'green', 'darkgreen']}, '2016/2017 dNDVI');
Map.addLayer(dNDVI_2021_2016, {min:-1, max:1, palette:['pink', 'white', 'green', 'darkgreen']}, '2016/2021 dNDVI');


// ======= Display Fire Boundary ======= //


// Display the fire boundary
Map.addLayer(fire_perimeter.style({width: 2,
                                  color: "red",
                                  fillColor: "#00000000"}),{},"Fire Boundary", 1);


// [18:20] Bridget Hass (US)

// Plot histogram charts for each dNDVI image in the console.
var dNDVI_hist_2017_2016 = ui.Chart.image.histogram({image: dNDVI_2017_2016, region: studyArea, scale: 50})
    .setOptions({title: 'dNDVI Histogram, 2016/2017',
            hAxis: {title: 'dNDVI Value',titleTextStyle: {italic: false, bold: true},},
            vAxis: {title: 'Count', titleTextStyle: {italic: false, bold: true}},});

print(dNDVI_hist_2017_2016);

var dNDVI_hist_2021_2016 = ui.Chart.image.histogram({image: dNDVI_2021_2016, region: studyArea, scale: 50})
    .setOptions({title: 'dNDVI Histogram, 2016/2021',
            hAxis: {title: 'dNDVI Value',titleTextStyle: {italic: false, bold: true},},
            vAxis: {title: 'Count', titleTextStyle: {italic: false, bold: true}},});


print(dNDVI_hist_2021_2016);

<IPython.core.display.Javascript object>